# Actividad Integradora

Diego Antonio García Padilla (A01710777)

## Leer expresión regular

Leer la expresión regular, parsearla y convertirla a una expresión postfija.

In [403]:
# Leer alfabeto

alfabeto_input = input("Introduce el alfabeto: ")

alfabeto = []

# Extraer los símbolos del alfabeto
# O(n)

for simbolo in alfabeto_input:
    alfabeto.append(simbolo)
    
alfabeto = ''.join(alfabeto)

print("El alfabeto es: ", alfabeto)

El alfabeto es:  ab


In [404]:
# Leer expresión regular (se asume que es correcta)

regex_input = input("Ingrese la expresión regular: ")

print("La expresión regular es:", regex_input)


La expresión regular es: (a|b)*abb


In [405]:
# Parsear expresión regular
# Ejemplo: a*b(a|b) -> a*.b.(a|b)

# Formatear expresión regular
# O(n)

def format_regex(regex_input):
    regex = ""

    # Agregar punto en concatenación
    for i in range(len(regex_input)):
        if i < len(regex_input) - 1:
            if regex_input[i] in alfabeto and regex_input[i + 1] in alfabeto:
                regex += regex_input[i] + "."
            elif regex_input[i + 1] == "(":
                regex += regex_input[i] + "."
            elif regex_input[i] == ")" and regex_input[i + 1] in alfabeto:
                regex += regex_input[i] + "."
            else:
                regex += regex_input[i]
        else:
            regex += regex_input[i]

    # Agregar punto entre operadores de repetición y sus operandos
    op_rep = ["*", "+"]

    for i in range(len(regex)):
        if i < len(regex) - 1:
            if regex[i] in op_rep and regex[i + 1] in alfabeto:
                regex = regex[:i + 1] + "." + regex[i + 1:]
            elif regex[i] in op_rep and regex[i + 1] == "(":
                regex = regex[:i + 1] + "." + regex[i + 1:]

    # Retornar expresión regular formateada
    return regex

# Formatear expresión regular dos veces para asegurar que no queden puntos juntos
regex = format_regex(regex_input)

print("RegEx: ", regex)

RegEx:  (a|b)*.a.b.b


In [406]:
# Parsear expresión regular a posfijo
# O(n)

def regex_a_postfix(regex):
    postfix = ""
    stack = []
    
    # Definir la prioridad de los operadores
    prioridad = {'*': 3, '+': 2, '.': 1}
    
    for char in regex:
        # Si el caracter es un símbolo del alfabeto, agregarlo al posfijo
        if char in alfabeto:
            postfix += char
            
        # Si el caracter es un paréntesis izquierdo, agregarlo al stack
        elif char == "(":
            stack.append(char)
        
        # Si el caracter es un operador, agregarlo al stack 
        elif char == ")":
            # Sacar operadores del stack hasta encontrar el paréntesis izquierdo
            while stack[-1] != "(":
                postfix += stack.pop()
            # Sacar el paréntesis izquierdo
            stack.pop()
        else:
            while stack and stack[-1] != "(" and prioridad[char] <= prioridad[stack[-1]]:
                postfix += stack.pop()
            stack.append(char)
    
    while stack:
        postfix += stack.pop()
    
    return postfix

postfix = regex_a_postfix(regex)

print("Posfijo: ", postfix)

Posfijo:  ab|*a.b.b.


## Generar NFA

Generar un autómata finito no determinista a partir de la expresión regular postfija.

In [407]:
# Generar autómata finito no determinista

expresiones = []
transiciones = {}

estados = -1
estado_inicial = 0
estado_final = 0

# Generar autómata finito no determinista
# O(n)

for char in postfix:
    # Si es un símbolo del alfabeto, generar un autómata finito no determinista con un estado inicial,
    # un estado final y una transición con el símbolo.
    if char in alfabeto:
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transición vacía de estado inicial a estado final
        transiciones[estado_inicial] = []
        
        # Agregar transición de estado inicial a estado final con el símbolo
        transiciones[estado_inicial].append([estado_final, char])
    
    # Si es un operador de cerradura de Kleene
    if char == "*":
        # Tomar la última expresión generada (para repetición es necesario tener una expresión)
        [e1, e2] = expresiones.pop()
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        if transiciones.get(e2) is None:
            transiciones[e2] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append([e1, "#"])
        transiciones[e2].append([estado_final, "#"])
        transiciones[estado_inicial].append([estado_final, "#"])
        transiciones[e2].append([e1, "#"])
        
    # Si es un operador de cerradura abierta
    if char == "+":
        # Tomar la última expresión generada (para repetición es necesario tener una expresión)
        [e1, e2] = expresiones.pop()
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        if transiciones.get(e2) is None:
            transiciones[e2] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append([e1, "#"])
        transiciones[e2].append([estado_final, "#"])
        transiciones[e2].append([e1, "#"])
    
    # Si es un operador de concatenación, unir los dos últimos autómatas finitos no deterministas generados
    if char == ".":
        # Tomar las dos últimas expresiones generadas (para concatenación es necesario tener dos expresiones)
        [e1, e2] = expresiones.pop()  # 2do
        [e3, e4] = expresiones.pop()  # 1er
        
        # Agregar transición vacía
        if transiciones.get(e4) is None:
            transiciones[e4] = []
        
        # Agregar transiciones
        transiciones[e4].append([e1, "#"])
        
        # Agregar estados a expresiones
        expresiones.append([e3, e2])
        
        # Actualizar estado inicial y final
        estado_inicial = e3
        estado_final = e2
    
    # Si es un operador de elección
    if char == "|":
        # Tomar las dos últimas expresiones generadas (para elección es necesario tener dos expresiones)
        [e1, e2] = expresiones.pop()  # 2do
        [e3, e4] = expresiones.pop()  # 1er
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        if transiciones.get(e2) is None:
            transiciones[e2] = []
        if transiciones.get(e4) is None:
            transiciones[e4] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append([e1, "#"])
        transiciones[estado_inicial].append([e3, "#"])
        transiciones[e2].append([estado_final, "#"])
        transiciones[e4].append([estado_final, "#"])
        
# Añadir estado final al diccionario de transiciones
# transiciones[estado_final] = []

# Ordenar transiciones
transiciones = dict(sorted(transiciones.items()))

# Imprimir resultado

print("Estado inicial: ", estado_inicial)
print("Estado final: ", estado_final)

# Imprimir transiciones
# O(n)

for transicion in transiciones:
    print("Transición: ", transicion, " -> ", transiciones[transicion])

Estado inicial:  6
Estado final:  13
Transición:  0  ->  [[1, 'a']]
Transición:  1  ->  [[5, '#']]
Transición:  2  ->  [[3, 'b']]
Transición:  3  ->  [[5, '#']]
Transición:  4  ->  [[2, '#'], [0, '#']]
Transición:  5  ->  [[7, '#'], [4, '#']]
Transición:  6  ->  [[4, '#'], [7, '#']]
Transición:  7  ->  [[8, '#']]
Transición:  8  ->  [[9, 'a']]
Transición:  9  ->  [[10, '#']]
Transición:  10  ->  [[11, 'b']]
Transición:  11  ->  [[12, '#']]
Transición:  12  ->  [[13, 'b']]


## Convertir NFA a DFA

Convertir el autómata finito no determinista a un autómata finito determinista.

In [408]:
# Definir funciones

# E-Closure (Depth-First Search)
# Ver cuales son los estados a los que se puede llegar por medio de "#" (epsilon) desde el estado de entrada
# O(n^2)

def e_closure(estados, transiciones):
    # Inicializar variables
    stack = []
    e_closure = []
    
    # Agregar estados a la pila
    for estado in estados:
        stack.append(estado)
    
    # Recorrer pila
    while stack:
        estado = stack.pop()
        e_closure.append(estado)
        
        # Verificar si hay transiciones con epsilon
        if estado in transiciones:
            for transicion in list(transiciones[estado]):
                if transicion[1] == "#":
                    stack.append(transicion[0])
                
    return sorted(e_closure)

# Move (Depth-First Search)
# Ver cuales son los estados a los que se puede llegar por medio de un símbolo del alfabeto desde el conjunto de estados
# O(n^2)

def move(conjunto, simbolo, transiciones, estado_final):
    # Inicializar variables
    stack = []
    move = []
    
    # Recorrer conjunto de estados
    for estado in conjunto:
        # Si el estado es el estado final, no es necesario buscar más
        if estado == estado_final:
            break
        # Verificar si hay transiciones con el símbolo
        for transicion in transiciones[estado]:
            if transicion[1] == simbolo:
                stack.append(transicion[0])
                
    # Agregar estados a la pila
    while stack:
        estado = stack.pop()
        move.append(estado)
        
    return sorted(move)

In [410]:
# Generar autómata finito determinista

# Letra ASCII de la letra A
ascii_a = ord("A")

# Inicializar variables
estados_dfa = []                # Lista de conjuntos de estados
estados_aceptacion_dfa = []     # Lista de conjuntos de estados de aceptación
estado_inicial_dfa = chr(ascii_a) # Estado inicial
transiciones_dfa = {}           # Diccionario de transiciones
fila_transiciones = []          # Fila de transiciones 
fila_estados = []               # Fila de estados

# Calcular e-closure del estado inicial
estados = e_closure([estado_inicial], transiciones)
estados_dfa.append(estados)
estados_dfa.append([])

# Añadir transiciones del estado inicial a la fila de transiciones
transiciones_dfa[chr(ascii_a)] = estados_dfa

# Añadir estado a la fila de estados
fila_estados.append([chr(ascii_a), estados])

# Crear autómata finito determinista
while fila_estados:
    # Tomar el primer estado de la fila de estados
    [letra, estados] = fila_estados.pop(0)
    
    # Si el estado es de aceptación, añadirlo a la lista de estados de aceptación 
    if estado_final in estados:
        estados_aceptacion_dfa.append(letra)
    
    # Limpiar la fila de transiciones
    fila_transiciones = []
        
    # Recorrer el alfabeto
    for simbolo in alfabeto:
        # Inicializar variables
        estados_dfa = []    # Lista de conjuntos de estados
        encontrado = False   # Bandera para saber si el estado ya existe
        
        # Calcular move de los estados actuales con el símbolo actual
        move_actual = move(estados, simbolo, transiciones, estado_final)
        
        # Calcular e-closure del move actual
        e_closure_actual = e_closure(move_actual, transiciones)
        
        # Estados nuevos
        nuevos_estados = e_closure_actual
        
        # Validar en caso de que el e-closure actual sea vacío
        if not nuevos_estados:
            estados_dfa.append(estados)
            estados_dfa.append([])
            fila_transiciones.append("uwu")
            estados_dfa[1].append(fila_transiciones)
            transiciones_dfa[letra] = estados_dfa
        else:
            # Recorrer la lista de conjuntos de estados
            for key, valor in list(transiciones_dfa.items()):
                # Si el e-closure actual ya existe en la lista de conjuntos de estados
                if nuevos_estados == valor[0]:
                    # El estado ya existe
                    encontrado = True
                    
                    # Añadir los estados
                    estados_dfa.append(estados)
                    estados_dfa.append([])
                    fila_transiciones.append(key)
                    estados_dfa[1].append(fila_transiciones)
                    transiciones_dfa[letra] = estados_dfa
                
            # Si el estado no existe, añadirlo a la lista de conjuntos de estados
            if not encontrado:
                # Añadir los estados a la lista de conjuntos de estados
                estados_dfa.append(nuevos_estados)
                estados_dfa.append([])
                
                # Añadir transiciones al diccionario de transiciones
                transiciones_dfa[chr(ord(key) + 1)] = estados_dfa
                
                # Añadir estado a la fila de estados
                estados_dfa = []
                estados_dfa.append(estados)
                estados_dfa.append([])
                
                fila_transiciones.append(chr(ord(key) + 1))
                estados_dfa[1].append(fila_transiciones)
                transiciones_dfa[letra] = estados_dfa
                fila_estados.append([chr(ord(key) + 1), nuevos_estados])
            
# Imprimir resultado
for key, valor in list(transiciones_dfa.items()):
    lista = []
    i = 0
    for item in valor[1][0]:
        if item != "uwu":
            lista.append((item, alfabeto[i]))
        i += 1
    print(key, "=>", lista)

estados_aceptacion_dfa = sorted(estados_aceptacion_dfa)
estados_aceptacion = list(estados_aceptacion_dfa)
print("Estados de aceptación:", estados_aceptacion)


A => [('B', 'a'), ('C', 'b')]
B => [('B', 'a'), ('D', 'b')]
C => [('B', 'a'), ('C', 'b')]
D => [('B', 'a'), ('E', 'b')]
E => [('B', 'a'), ('C', 'b')]
Estados de aceptación: ['E']
